Play slices of the waveforms:

In [1]:
import torchaudio
from IPython.display import Audio

waveform, _ = torchaudio.load("data/librispeech_subset/84-121123-0001.wav")
sample_rate=16000

start_time=1.1
end_time=1.46

start_frame = int(start_time * sample_rate)
end_frame = int(end_time * sample_rate)

waveform_slice = waveform[:, start_frame:end_frame]

Audio(waveform_slice.squeeze(), rate=16000)

Get the words & their indices for a .textgrid file from Librispeech

In [3]:
from pathlib import Path
import textgrids

    

input_dir = Path("data/librispeech_subset_alignments")
files = list(input_dir.rglob("*.TextGrid"))
output_path = "data/librispeech_subset_alignments/words_and_indices.txt"

with open(output_path, "w", encoding="utf-8") as f:
    for file in files:
        grid = textgrids.TextGrid(file)

        words_tier = grid["words"]
        file_name = file.stem
        print(file_name)
        
        f.write(f"{file_name}:\n")
        for idx, interval in enumerate(words_tier, start=1):
            if idx == 1:
                continue
            f.write(f"{idx-2}: {interval.text}\n")
        f.write("\n")


174-50561-0005
251-118436-0002
84-121123-0000
84-121123-0001
251-118436-0004
251-118436-0003
84-121123-0002
174-50561-0004
174-50561-0002
251-118436-0000
174-50561-0001
174-50561-0000
174-50561-0003
251-118436-0001
